In [13]:
%classpath add jar ../../konduit.jar

## Model Link
The dl4j model is taken from the dl4j-example here: https://github.com/eclipse/deeplearning4j-examples/blob/master/mvn-project-template/src/main/java/org/deeplearning4j/examples/sample/LeNetMNIST.java

In [30]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import io.micrometer.prometheus.PrometheusConfig;
import io.micrometer.prometheus.PrometheusMeterRegistry;
import io.micrometer.core.instrument.Counter;
import io.micrometer.core.instrument.binder.jvm.JvmMemoryMetrics;
import io.micrometer.core.instrument.binder.system.ProcessorMetrics;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

public class PrometheusEndpoint implements Endpoint {

    public static PrometheusMeterRegistry registry;
    public static List<Counter> classCounterIncrement = new ArrayList();
    public static List<String> labels = Arrays.asList("UnderWeight", "Normal_Range", "OverWeight", "Obese_ClassI", "Obese_ClassII", "Obese_ClassIII", "Obese_ClassIV");
    
    static {
        registry = new PrometheusMeterRegistry(PrometheusConfig.DEFAULT);
        
        if (registry != null) {
            System.out.println("Using metrics registry " + registry.getClass().getName() + " for inference");
            new JvmMemoryMetrics().bindTo(registry);
            new ProcessorMetrics().bindTo(registry);
            
            Counter serverUpTimeCounter = registry.counter("server.up.time");
            double increment = 5.0;
            new Timer().schedule(new TimerTask() {
                @Override
                public void run() {
                    serverUpTimeCounter.increment(increment);
                }
            }, 5000, ((int) increment) * 1000);
            
            
            for (String label : labels) {
                classCounterIncrement.add(Counter.builder(label)
                        .description("Classification counts seen so far for class label: " + label)
                        .baseUnit("classification.outcome")
                        .register(registry));
            }
        } else {
            System.out.println("Not using metrics registry.");
        }
    }
    
    private PipelineExecutor pipelineExecutor;

    public PrometheusEndpoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/bmi-metrics"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("text/plain; version=0.0.4; charset=utf-8"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> handler.response().putHeader(HttpHeaders.CONTENT_TYPE, "text/plain; version=0.0.4; charset=utf-8").end(registry.scrape());
    }
}

ai.konduit.PrometheusEndpoint

In [31]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import io.micrometer.core.instrument.Counter;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    private Counter requestsHandled = PrometheusEndpoint.registry.counter("requests.handled");

    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                handler.response().end(ObjectMappers.toJson(exec));
                taskHandler.complete();
                
                requestsHandled.increment();
                
                String bmiClass = exec.getString("bmi_class");
                int index = PrometheusEndpoint.labels.indexOf(bmiClass.replace(" ", "_").trim());
                System.out.format("BMI CLASS: %s, for index %s", bmiClass, index);
                if(index != -1) {
                    PrometheusEndpoint.classCounterIncrement.get(index).increment();
                }
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [32]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {
    
    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor), new PrometheusEndpoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [33]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("Saved %s at: %s%n", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker7190509432685184405/outDir
/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
Saved /tmp/beaker7190509432685184405/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar at: /home/shams/PycharmProjects/konduit-serving-demo/demos/6-bmi-onnx-pytorch/classpath


null

In [6]:
%%bash
java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id bmi-onnx-pytorch -c bmi-onnx-pytorch.yaml -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker2218157606244225458/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
INFO: Running command /home/shams/miniconda3/envs/beakerx/jre/bin/java -Dkonduit.logs.file.path=/home/shams/.konduit-serving/command_logs/bmi-onnx-pytorch.log -Dlogback.configurationFile=/tmp/logback-run_command_d2edc148b58a4eb7.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c bmi-onnx-pytorch.yaml -Dserving.id=bmi-onnx-pytorch
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs bmi-onnx-pytorch'
Unable to identify last server data file cleanup check
java.text.ParseException: Unparseable date: "11/16/20, 7:39 PM"
	at java.text.DateFormat.parse(DateFormat.java:366)
	at ai.konduit.serving.cli.launcher.LauncherUtils.cleanServerDataFilesOnceADay(LauncherUtils.java:170)
	at ai.kond

In [14]:
%%bash
konduit logs bmi-onnx-pytorch -l 100

20:11:11.541 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /home/shams/PycharmProjects/konduit-serving-demo/demos/6-bmi-onnx-pytorch/bmi-onnx-pytorch.yaml
20:11:11.547 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
20:11:11.548 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
20:11:11.549 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'bmi-onnx-pytorch'
20:11:11.911 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 27 PipelineStepRunnerFactory instances
20:11:12.311 [vert.x-worker-thread-0] INFO  a.k.serving.python.PythonRunner - Over riding python path :/home/shams/miniconda3/envs/beakerx/lib/python37.zip:/home/shams/miniconda3/envs/beakerx/lib/python3.7:/home/shams/miniconda3/envs/beakerx/lib/python3.7/lib-dynload:/home/shams/miniconda3/envs/beakerx/

In [8]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | bmi-onnx-pytorch               | inference  | 0.0.0.0:8082         | 402995  | started    




In [4]:
%%html
<img src="image_me.jpg"/>

In [12]:
%%bash
konduit inspect bmi-onnx-pytorch -q {port}

8082



In [12]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -X POST -F "image=@image_me.jpg" http://localhost:$(konduit inspect bmi-onnx-pytorch -q {port})/infer

{
  "bmi_value" : 22.18,
  "bmi_class" : "Normal_Range",
  "boxes" : [ 447.0, 174.0, 636.0, 470.0 ]
}



In [ ]:
%%bash
konduit stop bmi-onnx-pytorch